In [1]:
import pandas as pd
import numpy as np
from plotly import graph_objects as go
import plotly.express as px

In [2]:
home_page = pd.read_csv("data/home_page_table.csv")
search_page = pd.read_csv("data/search_page_table.csv")
payment_page = pd.read_csv("data/payment_page_table.csv")
payment_confirmation = pd.read_csv("data/payment_confirmation_table.csv")
user = pd.read_csv("data/user_table.csv")

In [16]:
home_page.head()

,user_id,page
0,313593,home_page
1,468315,home_page
2,264005,home_page
3,290784,home_page
4,639104,home_page


In [8]:
user["date"] = pd.to_datetime(user["date"])

In [12]:
user.head()

,user_id,date,device,sex
0,450007,2015-02-28,Desktop,Female
1,756838,2015-01-13,Desktop,Male
2,568983,2015-04-09,Desktop,Male
3,190794,2015-02-18,Desktop,Female
4,537909,2015-01-15,Desktop,Male


| user_id | date | device | sex | home_page | search_page | payment_page | payment_confirmation |
|---------|------|--------|----|----|----|-----|-----|
| 123 | 1970-01-01 | symbian | r | 1 | 1 | 0 | 0 |

In [21]:
df = (user
      .merge(home_page, how="left", on="user_id")
      .rename(columns={"page": "home_page"})
      .merge(search_page, how="left", on="user_id")
      .rename(columns={"page": "search_page"})
      .merge(payment_page, how="left", on="user_id")
      .rename(columns={"page": "payment_page"})
      .merge(payment_confirmation, how="left", on="user_id")
      .rename(columns={"page": "payment_confirmation"})
      .fillna(0))

In [24]:
pages = ["home_page", "search_page", "payment_page", "payment_confirmation"]

df[pages] = np.where(df[pages] == 0, 0, 1)

In [27]:
df.head()

,user_id,date,device,sex,home_page,search_page,payment_page,payment_confirmation
0,450007,2015-02-28,Desktop,Female,1,0,0,0
1,756838,2015-01-13,Desktop,Male,1,0,0,0
2,568983,2015-04-09,Desktop,Male,1,1,0,0
3,190794,2015-02-18,Desktop,Female,1,1,0,0
4,537909,2015-01-15,Desktop,Male,1,0,0,0


# Funnel

In [31]:
fig = go.Figure(go.Funnel(y=pages,
                          x=df[pages].sum().values,
                          textinfo="value+percent initial"))
fig.show()

# Funnel split by device

In [36]:
df_by_device = df.groupby("device")[pages].sum()
df_by_device

,home_page,search_page,payment_page,payment_confirmation
device,,,,
Desktop,60200,30100,3010,150
Mobile,30200,15100,3020,302


In [37]:
fig = go.Figure()

for device in ["Desktop", "Mobile"]:
    fig.add_trace(go.Funnel(name=device,
                            y=pages,
                            x=df_by_device.loc[device].values,
                            textinfo="value+percent initial"))
    
fig.show()